In [ ]:
import plotly.express as px
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import sweetviz as sv
px.defaults.template = 'seaborn'
px.defaults.width = 700
px.defaults.height = 500
pd.set_option('display.max_columns', 30)

In [ ]:
df = pd.read_excel('INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.xls',
                   sheet_name='INX_Future_Inc_Employee_Perform')

In [ ]:
category_orders = {'PerformanceRating': [2, 3, 4]}
df.head()

In [ ]:
my_report = sv.analyze(df)
my_report.show_html()

In [ ]:
fig = px.pie(df.sample(frac=1), names='PerformanceRating', color='PerformanceRating', hole=0.3,
             title='Performance Rating Distribution')
fig.show()
fig = px.histogram(df, x='PerformanceRating', color='PerformanceRating',
                   title='Number of data observations', category_orders=category_orders)
fig.show()

In [ ]:
cols_numerical = ['Age', 'DistanceFromHome', 'EmpEducationLevel', 'EmpEducationLevel', 'EmpEnvironmentSatisfaction',
                  'EmpHourlyRate','EmpJobInvolvement', 'EmpJobLevel', 'EmpJobSatisfaction', 'NumCompaniesWorked',
                  'EmpLastSalaryHikePercent', 'EmpRelationshipSatisfaction', 'TotalWorkExperienceInYears',
                  'TrainingTimesLastYear', 'EmpWorkLifeBalance', 'ExperienceYearsAtThisCompany',
                  'ExperienceYearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

for col in cols_numerical:
    fig = px.histogram(df, color='PerformanceRating', x=col, title=f'Relationship with {col}',
                       barnorm='percent', category_orders=category_orders)
    fig.show()
    fig = px.histogram(df, color='PerformanceRating', x=col, title=f'Relationship with {col}',
                       category_orders=category_orders)
    fig.show()
    fig = px.violin(df, x=col, color='PerformanceRating',
                   category_orders=category_orders, title=f'Relationship with {col}')
    fig.show()

In [ ]:
cols_categorical = ['Gender', 'EducationBackground','MaritalStatus','EmpDepartment','EmpJobRole',
                    'BusinessTravelFrequency', 'OverTime', 'Attrition']

for col in cols_categorical:
    fig = px.histogram(df, x=col, color='PerformanceRating', barnorm='percent',
                       category_orders=category_orders, title=f'Relationship with {col}')
    fig.show()
    fig = px.histogram(df, x=col, color='PerformanceRating',
                   category_orders=category_orders, title=f'Relationship with {col}')
    fig.show()

In [ ]:
from scipy.stats import chi2_contingency
#H0 is No correlation
#Lower P Rejection of H0 and association is proven
df_p_values_cat_cols = pd.DataFrame(columns=['pvalues'], index=cols_categorical)
for col in cols_categorical:
    stat, p, dof, expected = chi2_contingency(pd.crosstab(df['PerformanceRating'], df[col]).values)
    df_p_values_cat_cols.at[col, 'pvalues'] = p
px.bar(df_p_values_cat_cols, x=df_p_values_cat_cols.index, color=df_p_values_cat_cols.index, y='pvalues',
       title='chi squared test for correlation')

In [ ]:
corr_mat = df[cols_numerical + ['PerformanceRating']].corr()
sns.heatmap(corr_mat)

In [ ]:
corr_columns = [i for i in corr_mat.columns if i not in ['PerformanceRating']]
corr_df = corr_mat.loc[corr_columns,['PerformanceRating', 'Age']]
px.bar(corr_df,x=corr_df.index, color=corr_df.index,y = 'PerformanceRating',
       title='correlation coeff for numerical features', height=690).update_yaxes(title_text='correlation_coefficient')

In [ ]:
corr_mat.loc['PerformanceRating', :].plot.bar()

In [ ]:
corr_mat.loc[corr_columns,['PerformanceRating', 'Age']]

In [ ]:
corr_mat_one_side = corr_mat.copy()
for i in range(len(corr_mat_one_side)):
    for j in range(i):
        corr_mat_one_side.iat[i, j] = 0
corr_mat_one_side

filt1 = corr_mat_one_side.melt(ignore_index=False)['value'] != 1
filt2 = corr_mat_one_side.melt(ignore_index=False)['value'] != 0
filt = filt1 & filt2
corr_mat_one_side.melt(ignore_index=False)[filt].sort_values('value', ascending=False).head(20)

In [ ]:
pd.crosstab(df['EmpDepartment'], df['EmpJobRole'])